# Inverting s and o in RDFs

In [1]:
from sentence_transformers import SentenceTransformer
import datasets
from dataset_wrappers import invert_all_triples
from functools import partial
import torch
from torch.nn import functional as F
from multiprocess import set_start_method

/home/teven/virtualenvs/dev_transformers/lib/python3.8/site-packages/jax/__init__.py:27: UserWarning: cloud_tpu_init failed: ConnectionError(MaxRetryError("HTTPConnectionPool(host='metadata.google.internal', port=80): Max retries exceeded with url: /computeMetadata/v1/instance/attributes/agent-worker-number (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f01c6befcd0>: Failed to establish a new connection: [Errno -2] Name or service not known'))"))
 This a JAX bug; please report an issue at https://github.com/google/jax/issues
  _warn(f"cloud_tpu_init failed: {repr(exc)}\n This a JAX bug; please report "


In [2]:
trex = datasets.load_dataset("json", data_files="../outputs/dataset_embeddings/from_all/trex.jsonl")

Using custom data configuration default-dabe789c260fa606
Reusing dataset json (/home/teven/.cache/huggingface/datasets/json/default-dabe789c260fa606/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
model = SentenceTransformer("../exported_outputs/large_bs_runs/all_datasets_bs2560_hard_neg-2022-01-13_04-09-51/checkpoints/7032")

In [4]:
trex = trex.shuffle(seed=1951)

Loading cached shuffled indices for dataset at /home/teven/.cache/huggingface/datasets/json/default-dabe789c260fa606/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b/cache-db090eb5c317ea20.arrow


In [5]:
subset = trex["train"].select(range(10000))

In [6]:
subset = subset.map(partial(invert_all_triples, rdf_key="triples"))

Loading cached processed dataset at /home/teven/.cache/huggingface/datasets/json/default-dabe789c260fa606/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b/cache-bebf0934a935fd92.arrow


In [10]:
subset = subset.filter(lambda x: len(x["triples"]) == 1)

  0%|          | 0/10 [00:00<?, ?ba/s]

In [12]:
subset["triples"]

[[['Laertes', 'child', 'Odysseus']],
 [['Don Cardwell', 'position played on team / speciality', 'pitcher']],
 [['Kulabyte', 'headquarters location', 'San Marcos, Texas']],
 [['Daggers (seaQuest DSV)', 'part of the series', 'UEO']],
 [['Kotobeppu Yōhei', 'date of birth', '17 October 1965']],
 [['rice pudding', 'made from material', 'cinnamon']],
 [['Aar River', 'country', 'Switzerland']],
 [['The Atlas of Pern', 'genre', 'Atlas']],
 [['Lützenkirchen', 'country of citizenship', 'Germany']],
 [['Otmar Gazzari', 'member of sports team', 'Hajduk Split']],
 [['Wolfgang Friedmann', 'employer', 'Columbia University']],
 [['Katie Smith', 'award received', 'WNBA Finals MVP']],
 [['Mike York', 'member of sports team', 'Iserlohn Roosters']],
 [['3-Deoxy-D-arabino-heptulosonic acid 7-phosphate', 'has part', 'carbon']],
 [['Europe', 'shares border with', 'Africa']],
 [['Late Orchestration', 'follows', 'Late Registration']],
 [['Katarina Zivkovic', 'date of birth', '2 July 1989']],
 [['Flaming Youth 

In [9]:
subset["rdf_inverted"]

[[['Odysseus', 'child', 'Laertes']],
 [['Labor', 'member of political party', 'Edmond Hogan'],
  ['Labor', 'member of political party', 'Edmond Hogan']],
 [['Harvard University', 'educated at', 'Maya Jasanoff'],
  ['Harvard University', 'employer', 'Maya Jasanoff'],
  ['Harvard University', 'educated at', 'She'],
  ['Harvard University', 'employer', 'She']],
 [['Birjand', 'place of birth', 'Al-Birjandi'],
  ['Iran', 'country of citizenship', 'Al-Birjandi'],
  ['Husayn', 'child', 'Ali'],
  ['Ali', 'relative', 'Muhammad'],
  ['Ali', 'father', 'Husayn'],
  ['Iran', 'country', 'Birjand'],
  ['Persian', 'official language', 'Iran']],
 [['pitcher', 'position played on team / speciality', 'Don Cardwell']],
 [['journalist', 'occupation', 'Elizabeth Becker'],
  ['journalist', 'occupation', 'Elizabeth Becker'],
  ['Pulitzer Prize', 'award received', 'New York Times'],
  ['Pulitzer Prize', 'award received', 'New York Times']],
 [['Hangul', 'original language of film or TV show', 'Syndrome (TV ser

In [ ]:
def pair_sims_datasets_map(examples: dict, rank: int = 0, model: SentenceTransformer = None, text_key: str = "text", rdf_key: str = "rdf_linearized", batch_size: int = 16, similarity_key: str = "similarity"):
    device = f"cuda:{rank % torch.cuda.device_count()}"
    model = model.to(device)
    embeddings1 = torch.stack(
        model.encode(examples[text_key], show_progress_bar=False, convert_to_numpy=False, batch_size=batch_size, device=device))
    embeddings2 = torch.stack(
        model.encode(examples[rdf_key], show_progress_bar=False, convert_to_numpy=False, batch_size=batch_size, device=device))

    cos_sims = F.cosine_similarity(embeddings1, embeddings2).detach().cpu().numpy()
    examples[similarity_key] = cos_sims

    return examples

In [ ]:
subset.map(
    partial(pair_sims_datasets_map, model=model, text_key="text", rdf_key="rdf_inverted", batch_size=16, similarity_key="inverted_similarity"), batched=True, batch_size=16, num_proc=2)

In [ ]:
graph_lengths = [len(graph) for graph in subset["triples"]]

In [ ]:
filtered_graph_lengths = [len(graph) for graph in subset.filter(lambda x: x["similarity"] > 0.7)["triples"]]

In [ ]:
subset.filter(lambda x: x["similarity"] > 0.7)["similarity"]

# Nothing to do with inversion, counting graph lengths before and after filtration

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
np.mean(filtered_graph_lengths)

In [ ]:
np.mean(graph_lengths)

In [ ]:
plt.hist(filtered_graph_lengths, color='blue', edgecolor='black',
         bins=100)

In [ ]:
plt.hist(graph_lengths, color='blue', edgecolor='black',
         bins=100)

In [ ]:
max(filtered_graph_lengths)

In [ ]:
rejected_one_triples = subset.filter(lambda x: x["similarity"] < 0.7 and len(x["triples"])  == 1)

In [ ]:
for example in rejected_one_triples:
    print(example["triples"])
    print(example["text"])
    print(example["similarity"])
    print()
    print()